In [ ]:
%pip install requests bs4 

In [2]:
import json
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse


def get_html_with_requests(url):
  # Extrair domínio e caminho da URL
    url = url if url.startswith("https://") else f"https://{url}"
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    headers = {
      
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6,es;q=0.5',
        'Cookie': 'ajs_anonymous_id=a03ba2e0-6658-4732-bca0-5c82a729610d; blueID=158f8c36-f884-4eeb-9dd2-41ddb36954f2; _fbp=fb.2.1719565497536.945089496421286543; _tt_enable_cookie=1; _ttp=yDIqAjbcrTkStMgfYKWUtQqV7qI; _scid=fe8f87fb-9164-4128-931f-6db8349a5902; _pin_unauth=dWlkPU16QmxZV015TVRndE16QmhOUzAwWmpRekxUZzROakF0Wm1VeFlUSTROREE1WXpJeg; _hjSessionUser_257734=eyJpZCI6IjZlYmVkNTczLWRlNDItNTljNy1iYjBkLWRjMzQxYzg1ZThkZiIsImNyZWF0ZWQiOjE3MTk1NjU0OTc1ODksImV4aXN0aW5nIjp0cnVlfQ==; __utmz=other; blueULC=blue; OptanonAlertBoxClosed=2024-06-28T10:07:39.896Z; _gac_UA-58792402-1=1.1719578340.CjwKCAjwvvmzBhA2EiwAtHVrbx5PuZ-Y6ZmDRgxSQAE3bhfdMkfQ6qO_h_InopnIh9t314DXR2UOdhoCJPMQAvD_BwE; origem=adwords; _rtbhouse_source_=AdWords; anonymousClient=true; _gcl_aw=GCL.1719608537.CjwKCAjwvvmzBhA2EiwAtHVrb7a4Gf1uUaMKzAcQ623DOKsXA0ynRueURMbfhKPxXrTvmq46s3W5ZBoCXV8QAvD_BwE; _gcl_gs=2.1.k1$i1719608535; _gid=GA1.3.1821486782.1720470235; _ScCbts=%5B%5D; petzCarrinho=true; JSESSIONID=9E56A4F6251336BC2EEC9640328DF1CE; _hjSession_257734=eyJpZCI6ImRmYjFhNzkxLTcwMDAtNDc5ZC1hZDA2LTE4NGYwZGMwOTQ3MSIsImMiOjE3MjA2MzEyODk1MzEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; OptanonConsent=isGpcEnabled=0&datestamp=Wed+Jul+10+2024+14%3A33%3A28+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=202404.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=5d44d058-e0ca-4634-af6a-6ae970855f35&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CC0002%3A1%2CC0004%3A1&AwaitingReconsent=false&intType=1&geolocation=BR%3BRJ; AWSALB=hZab3iIPYsD0MIUknEJjbESldllV2XbXHeW6ABubXDqQC1w+fB9ShoCYGJIR2z0ndrcA5n/bdu5qP2SU+sm1uOdvT/LotUC6UCm2meSI5Mm4DIVUR9N7D5oqkEID; AWSALBCORS=hZab3iIPYsD0MIUknEJjbESldllV2XbXHeW6ABubXDqQC1w+fB9ShoCYGJIR2z0ndrcA5n/bdu5qP2SU+sm1uOdvT/LotUC6UCm2meSI5Mm4DIVUR9N7D5oqkEID; _ga_JB2JHD7FCJ=GS1.1.1720631306.23.1.1720632809.57.0.0; _ga=GA1.1.604824160.1719565115; _uetsid=006ad4a03d6811ef82e9db6f4cebf44a; _uetvid=7e097870352d11ef8996917c57f63d87; _scid_r=fe8f87fb-9164-4128-931f-6db8349a5902; _derived_epik=dj0yJnU9OVpXTXl0T0xlTmRISjQ3aGwtVFQ1SWZjVmJuT0JiaFkmbj1zc3Z6NkNFQXNmdnZHWmFDdEpIX093Jm09MSZ0PUFBQUFBR2FPeGVzJnJtPTEmcnQ9QUFBQUFHYU94ZXMmc3A9Mg; cto_bundle=Eb4Xml9POEdtRUtaTE12dTVMYjVqT2xtMDIxaDE5blJXQWRHRkZXVEJlcUdNOTdFRGNwdGtMeWslMkZGVnBENzMwM0JYMDVmQkdIeVQ4NW1qMW9laEslMkZRQnl3dzElMkZPWFUlMkZmTFJxNGdLeXUlMkZTMTRqZyUyRndXSFZ4dEsxS1hyMk1ZeGpOQVU4bmU1bkFZY1lwcHp2M3NVZ1FWQWNmYmRYS0RIc1N1ZzFBcWRZcWNlOVcxRkVaeVZGd0YzQUxFMWglMkZheXhJa0cyJTJGdUg0NiUyQk92VCUyRnVZZmtxQWN6QVAlMkZzdyUzRCUzRA; ab.storage.sessionId.8160f757-f57f-4d17-9cfb-44c82d5e8f64=%7B%22g%22%3A%221c387921-d350-d759-9bb8-04a9e46b2359%22%2C%22e%22%3A1720632840493%2C%22c%22%3A1720632809379%2C%22l%22%3A1720632810493%7D; RT="z=1&dm=www.petz.com.br&si=92ca62e5-ee3d-405f-ab43-07f1cffe3709&ss=lyg3fxm8&sl=6&tt=cia&obo=2&rl=1&ld=zeiw&r=ke1m2ztx&ul=zeix"',
        'Priority': 'u=0, i',
        'Referer': f"{domain}/marcas",
        'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        print(f"HTML obtido com sucesso para a URL: {url}")
        return response.text
    except Exception as e:
        print(f"Erro ao obter HTML com requests: { url}")
        return None
 
    
   
  


In [8]:





def extract_images(html_content: str):
    images = []
    try:
        if not html_content:
            print("Conteúdo HTML vazio ou inválido.")
            return []

        # Analisar o conteúdo HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        # Extrair imagens da div 'swiper-wrapper'
        swiper_div = soup.find('div', {'class': 'swiper-wrapper'})

        if swiper_div:
            
             print("Div 'swiper-wrapper' encontrada")
             tagsImg= swiper_div.find_all('img')
             print( tagsImg)
             for img_tag in tagsImg:
                img_url = img_tag.get('src')
                if img_url:
                 images.append(img_url)
                 print(f"Link de imagem encontrada: {img_url}")
        else:
            print("Div 'swiper-wrapper' não encontrada")
        
        print(images)
        return images
    except Exception as e:
        print(f"Erro ao extrair imagens: {e}")
        return []
    

def get_images(produto: dict):
    variations = produto['variations']
    complete_variations = []
    link_default_images = []
    unique_images = set()
    all_images = []
    print(f"Iniciando a extração de imagens para o produto: {variations} (ID: {produto['id']})")
    
    try:
        # Extrair imagens das variações
        for variation in variations:
            print(f"Iniciando a extração de imagens da variação {variation['name']}")
            link = variation['link']
            html_content = get_html_with_requests(link)
            if not html_content:
                print(f"Falha ao obter HTML para a variação {variation['name']}")
                continue
            
            variation_images = extract_images(html_content)
            print(f"Imagens extraídas da variação {variation['name']}: {variation_images}")
            variation['linkImages'] = list(variation_images)  # Inserir links de variação nas variações
            all_images.extend(variation_images)  # Coletar todos os links de imagens
        
        # Verificar e mover imagens repetidas
        for img_url in all_images:
            if img_url in unique_images:
                link_default_images.append(img_url)
                print(f"Imagem repetida encontrada e movida para link_default_images: {img_url}")
            else:
                unique_images.add(img_url)
                complete_variations.append(img_url)
                print(f"Imagem única encontrada e adicionada a complete_variations: {img_url}")
        
        # Remover imagens repetidas das variações
        for variation in variations:
            variation['linkImages'] = [img for img in variation['linkImages'] if img not in link_default_images]
        
        # Verificando imagens repetidas em link_default_images e removendo duplicatas
        link_default_images = list(set(link_default_images))
        
        print(f"link_default_images: {link_default_images}")
        print(f"complete_variations: {complete_variations}")
        return link_default_images, complete_variations
    except Exception as e:
        print(f"Erro ao extrair imagens: {e}")
        return [], []  # Retornar uma tupla vazia em caso de erro



In [ ]:


def format_images(product: dict, link_default_images: list, complete_variations: list):
    product_id = product['id']
    print(f"Iniciando a formatação de imagens para o produto: {product['name']} (ID: {product['id']})")
    try:
        formatted_images = {
            "imagesDefault": [f"{product['id']}_{idx}.jpg" for idx in range(len(link_default_images))],
            "variations": []
        }
        print(f"Imagens padrão formatadas: {formatted_images['imagesDefault']}")

        for idx, images in enumerate(complete_variations):
            formatted_images["variations"].append({
                "images": [f"{product_id}_variation_{idx}_{img_idx}.jpg" for img_idx in range(len(images))]
            })
        
        return formatted_images
    except Exception as e:
        print(f"Erro ao formatar imagens: {e}")
        return {}



def count_brands_and_products(json_file_path):
    print("Contando marcas e produtos no arquivo")
    
    if not os.path.exists(json_file_path):
        print(f"Arquivo {json_file_path} não existe.")
        return

    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print(f"Arquivo {json_file_path} carregado com sucesso.")
    
    total_brands = len(data['marcas'])
    total_products = sum(len(marca['produtos']) for marca in data['marcas'])
    
    print(f"Total de marcas: {total_brands}")
    print(f"Total de produtos: {total_products}")





def save_image_links(json_file_path):
    print("Iniciando a função save_image_links")
    
    if not os.path.exists(json_file_path):
        print(f"Arquivo {json_file_path} não existe.")
        return

    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print(f"Arquivo {json_file_path} carregado com sucesso.")
    
    products_with_missing_variation_names = []

    for marca in data['marcas']:
        print(f"Processando marca: {marca['name']}")
        
        for produto in marca['produtos']:
            print(f"Processando produto: {produto['name']} (ID: {produto['id']})")
            missing_fields = [field for field in ['id', 'variations', 'link'] if field not in produto]
            if missing_fields:
                print(f"Produto {produto.get('name', 'desconhecido')} está faltando informações essenciais: {', '.join(missing_fields)}. Pulando.")
                continue
            
            print(f"Processando produto: {produto['name']} (ID: {produto['id']})")
            product_id = produto['id']
            
            # Obter imagens do produto
            link_default_images, complete_variations = get_images(produto)
            print(f"Imagens extraídas para o produto: {produto['name']} (ID: {product_id})")
            
            # Formatar as imagens extraídas
            formatted_images = format_images(produto, link_default_images, complete_variations)
            print(f"Imagens formatadas para o produto: {produto['name']} (ID: {product_id})")
            
            # Mesclar as novas informações de imagens com as existentes
            if 'imagesDefault' in produto:
                produto['images']['links'].extend(formatted_images['imagesDefault'])
            else:
                produto['images']['links'] = formatted_images['imagesDefault']
            
            for variation in produto['variations']:
                if 'name' not in variation:
                    print(f"Variação sem nome encontrada no produto {produto['name']} (ID: {produto['id']}). Pulando.")
                    products_with_missing_variation_names.append(produto['name'])
                    continue
                for new_variation in formatted_images['variations']:
                    if 'name' in new_variation and variation['name'] == new_variation['name']:
                        if 'linkImages' in variation:
                            variation['linkImages'].extend(new_variation['images'])
                        else:
                            variation['linkImages'] = new_variation['images']
            
            print(f"Informações de imagens atualizadas para o produto: {produto['name']} (ID: {product_id})")
    
    with open(json_file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    print(f"Links de imagens salvos no arquivo {json_file_path}")

    if products_with_missing_variation_names:
        print("Produtos com variações sem a chave 'name':")
        for product_name in products_with_missing_variation_names:
            print(f"- {product_name}")

# Exemplo de chamada das funções

count_brands_and_products('marcas_selecionadas.json')
save_image_links('marcas_selecionadas.json')



